# 01 - Data Scraping

Este notebook tem como objetivo desenvolver um scraper para coletar dados das notas obtidas na faculdade, incluindo as médias e as modas das notas dos alunos que cursaram as mesmas disciplinas.

## Importações

In [1]:
# Bibliotecas padrão
import os
import re
import time
import platform
from abc import ABC, abstractmethod
from urllib.parse import urlparse
from typing import Literal, Tuple, Iterable

# Bibliotecas de terceiros
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Constantes e sets

In [2]:
OPTIONS = Options()  # Utilizado para determinar as opções do driver
OPTIONS.add_argument('--disable-gpu')  # Desabilita a aceleração de GPU (opcional)
OPTIONS.add_argument('--window-position=0,-1800')  # Para mover para o monitor de cima
OPTIONS.add_argument('--start-maximized')  # Deixa em tela cheia
# OPTIONS.add_argument('--headless')  # Ativa o modo headless (não mostra a tela do browser)

MAX_TIME_TO_WAIT = 60  # Tempo máximo em segundos para encontrar um elemento nas páginas

load_dotenv();

## Scripts

Antes de partir para a construção do Scraper, vamos definir uma classe abstrata para garantir que todos os Scrapers definidos neste projeto implementem os mesmos métodos.

In [3]:
class BaseScraper(ABC):
    @abstractmethod
    def __init__(self, options: Options, credentials: dict, max_time_to_wait: float) -> None:
        ...

    @abstractmethod
    def _setup_driver(self, options: Options) -> None:
        ...

    @abstractmethod
    def login(self) -> None:
        ...    

    @abstractmethod
    def get_grades_df(self) -> pd.DataFrame:
        ...

    @abstractmethod
    def shutdown(self) -> None:
        ...

### Scrapers para Sigaa

Feito isso, vamos inicialmente desenvolver a classe que vai instanciar os Scrapers que irão coletar as minhas notas. Como tanto as notas da graduação como da pós-graduação estão disponiveis no Sigaa de cada uma das faculdades, os Scrapers terão a mesma estrutura.

In [4]:
class SigaaScraper(BaseScraper):
    def __init__(self, options: Options, credentials: dict[Literal['username', 'password'], str], max_time_to_wait: float, sigaa_base_url: str) -> None:
        self._setup_driver(options)
        self.__credentials = credentials
        self.__max_time_to_wait = max_time_to_wait
        self.__sigaa_base_url = sigaa_base_url  # Exemplo: http://sigaa.ufpe.br/sigaa/


    def _setup_driver(self, options):
        self.__driver = webdriver.Chrome(options=options)


    def login(self) -> None:
        # Acessa a página do UFABC Next
        self.__driver.get(self.__sigaa_base_url)

        # Localiza o campo de usuário e senha
        input_username = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.presence_of_element_located((By.NAME, 'user.login'))
        )
        input_username.send_keys(self.__credentials['username'])
        input_pswd = self.__driver.find_element(by=By.NAME, value='user.senha')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
        input_pswd.send_keys(self.__credentials['password'])
        submit_button = self.__driver.find_element(By.XPATH, '//input[@type="submit" and @value="Entrar"]')
        submit_button.click()


    def _go_to_grades_page(self) -> None:
        # Seta a página base do "Portal do Discente" -> URL base seguida do path abaixo
        portal_url = urlparse(self.__sigaa_base_url)._replace(path='sigaa/portais/discente/discente.jsf').geturl()
        # Vai para a página "Portal do Discente" (passo a mais)
        self.__driver.get(portal_url)

        # Clica na aba "Ensino" e depois "Consultar Minhas Notas"
        dropdown = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.element_to_be_clickable((By.XPATH, '//span[@class="ThemeOfficeMainFolderText" and text()="Ensino"]'))
        )
        dropdown.click()
        get_grades_opt = self.__driver.find_element(by=By.XPATH, value='//td[@class="ThemeOfficeMenuItemText" and text()="Consultar Minhas Notas"]')
        get_grades_opt.click()


    def get_grades_df(self) -> pd.DataFrame:
        # Puxa os dados da tabela
        self._go_to_grades_page()

        # Puxa os dados da tabela
        tables_elements = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//table[@class="tabelaRelatorio"]'))
        )

        df_list = []
        for table in tables_elements:
            # Agora vamos extrair as linhas da tabela
            year = table.find_element(By.TAG_NAME, 'caption').text
            rows = table.find_elements(By.TAG_NAME, 'tr')

            # Extraindo os cabeçalhos (primeira linha da tabela)
            headers = [header.text for header in rows[0].find_elements(By.TAG_NAME, 'th')]

            # Extraindo as linhas de dados
            data = []
            for row in rows[1:]:  # pulando o cabeçalho
                columns = row.find_elements(By.TAG_NAME, 'td')
                row_data = [col.text for col in columns]
                data.append(row_data)

            # Convertendo os dados extraídos para um DataFrame do Pandas
            df_tmp = pd.DataFrame(data, columns=headers)
            df_tmp['Ano'] = year
            
            df_list.append(df_tmp)
        
        df = pd.concat(df_list)
        df = df[['Ano', 'Código', 'Disciplina', 'Resultado', 'Situação']]

        return df


    def shutdown(self):
        self.__driver.quit()
    

#### UFABC

Inicialmente, vamos começar com as notas da graduação e, em seguida, vamos para a pós-graduação.

In [5]:
credentials = {
    'username': os.environ['UFABC-USERNAME'],
    'password': os.environ['UFABC-PASSWORD']
}

ufabc_scraper = SigaaScraper(options=OPTIONS, credentials=credentials, max_time_to_wait=MAX_TIME_TO_WAIT, 
                             sigaa_base_url='http://sig.ufabc.edu.br/sigaa')
ufabc_scraper.login()
df_ufabc = ufabc_scraper.get_grades_df()
df_ufabc = df_ufabc[df_ufabc['Situação'] != 'CANCELADO'].sort_values('Ano', ascending=True)  # Para remover as matérias que foram canceladas
df_ufabc = df_ufabc.replace({'2020.QS': '2020.3'})
ufabc_scraper.shutdown()

df_ufabc

,Ano,Código,Disciplina,Resultado,Situação
5,2017.2,BIL0304-15,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,C,APROVADO
0,2017.2,BCS0001-15,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,A,APROVADO
1,2017.2,BIS0005-15,BASES COMPUTACIONAIS DA CIÊNCIA,A,APROVADO
4,2017.2,BIK0102-15,ESTRUTURA DA MATÉRIA,A,APROVADO
3,2017.2,BIS0003-15,BASES MATEMÁTICAS,C,APROVADO
...,...,...,...,...,...
1,2023.2,ESTA017-17,LABORATÓRIO DE MÁQUINAS ELÉTRICAS,A,APROVADO
0,2023.2,ESTA011-17,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,A,APROVADO
2,2023.3,ESTA904-17,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,A,APROVADO
1,2023.3,ESTA022-17,TEORIA DE ACIONAMENTOS ELÉTRICOS,A,APROVADO


Verificado que está tudo certo, podemos exportar os dados.

In [6]:
df_ufabc.to_csv('./data/notas-ufabc.csv', sep=';', index=False)  # Vamos usar ';' pois algumas matérias podem ter ',' no nome

Em seguida, vamos para a pós-graduação.

_Vale destacar que algumas matérias como "Termodinâmica I", "Sistemas Operacionais" e "Engenharia de Software" não constam na tabela acima devido a estas matérias terem sido cursadas durante o intercâmbio na Alemanha_.

#### CIn-UFPE

In [7]:
credentials = {
    'username': os.environ['CIN-USERNAME'],
    'password': os.environ['CIN-PASSWORD']
}

ufpe_scraper = SigaaScraper(options=OPTIONS, credentials=credentials, max_time_to_wait=MAX_TIME_TO_WAIT, 
                            sigaa_base_url='https://sigaa.ufpe.br/sigaa/')
ufpe_scraper.login()
df_ufpe = ufpe_scraper.get_grades_df()
ufpe_scraper.shutdown()

df_ufpe

,Ano,Código,Disciplina,Resultado,Situação
0,2024.1,CIN0068,APLICAÇÕES DE APRENDIZAGEM DE MÁQUINA,"9,9",APROVADO
1,2024.1,CIN0066,APRENDIZAGEM DE MÁQUINA I,"10,0",APROVADO
2,2024.1,CIN0067,APRENDIZAGEM DE MÁQUINA II,"10,0",APROVADO
3,2024.1,CIN0069,ARQUITETURA BIG DATA E ANALYTICS,"10,0",APROVADO
4,2024.1,CIN0061,ARQUITETURA DE SOFTWARE,"10,0",APROVADO
5,2024.1,CIN0065,ESTATÍSTICA AVANÇADA,"9,5",APROVADO
6,2024.1,CIN0071,FAMILIARIZAÇÃO AERONÁUTICA,"10,0",APROVADO
7,2024.1,CIN0064,INTRODUÇÃO A CIÊNCIA DE DADOS,"9,5",APROVADO
8,2024.1,CIN0070,PROJETO DE CIÊNCIA DE DADOS,"10,0",APROVADO
9,2024.1,CIN0062,QUALIDADE DE SOFTWARE,"10,0",APROVADO


Verificado que os dados estão certos, vamos exportá-lo de forma análoga ao que fizemos anteriormente.

In [8]:
df_ufpe.to_csv('./data/notas-ufpe.csv', sep=';', index=False)

### Scraper para UFABC Next

Por fim, vamos criar o Scraper que irá buscar as médias e as modas das notas dos alunos que cursaram a mesma matéria que eu na UFABC por meio de web scraping no site UFABC Next. Como não há algo semelhante para o CIn-UFPE, faremos isto apenas para UFABC.

In [9]:
class UFABCNextGradeScraper(BaseScraper):
    def __init__(self, options: Options, credentials: dict[Literal['e-mail', 'ra'], str], max_time_to_wait: float) -> None:
        self._setup_driver(options)
        self.__credentials = credentials
        self.__max_time_to_wait = max_time_to_wait


    @staticmethod
    def __standardize_name(name: str) -> Tuple[str, str]:
        name_std = name.lower().strip()
        name_splitted_list = re.split(r'[áéíóúàãõâêôüàç,.:/|]', name_std)  # Next está bugando com acento, então temos que remover e escrever a maior parte do texto sem acento para acharn
        name_splitted = sorted(name_splitted_list, key=lambda s: -len(s))[0]  # Pega a maior parte

        return name_std, name_splitted
    

    @staticmethod
    def print_grades_dict(grades_dict: dict) -> None:
        for grade, percentual in grades_dict.items():
            print(f'{grade}: {percentual}%')


    def _setup_driver(self, options: Options) -> None:
        self.__driver = webdriver.Chrome(options=options)


    def login(self) -> None:
        # Acessa a página do UFABC Next
        self.__driver.get('https://ufabcnext.com')

        # Localiza botão do Facebook e clica
        facebook_button = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'facebook-button'))
        )
        facebook_button.click()

        # Escreve o e-mail , RA e depois submete para fazer o login
        input_email = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.presence_of_element_located((By.ID, 'input-0'))
        )
        input_email.send_keys(self.__credentials['e-mail'])
        input_ra = self.__driver.find_element(by=By.ID, value='input-2')  # Não necessário usar o WebDriverWait -> Se já encontrou um, o outro tem que estar lá também 
        input_ra.send_keys(self.__credentials['ra'])
        submit_button = self.__driver.find_element(by=By.CSS_SELECTOR, value='button[type="submit"]')
        submit_button.click()
        time.sleep(2)  # Para garantir que está logado


    def get_subjects_professor_map(self) -> dict[Literal['professor', 'subject']]:
        # Vai para a página que tem o nome das matérias e professor(es)
        self.__driver.get('https://ufabcnext.com/app/history')

        # Puxa a tabela com os dados de Disciplina e professor(es)
        table = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.visibility_of_element_located((By.TAG_NAME, 'table'))
        )
        rows = table.find_elements(By.TAG_NAME, 'tr')  # Ordem é Disciplina, Professor de Teoria, Professor de Prática, Conceito e Créditos
        data = []
        for row in rows:
            columns = row.find_elements(by=By.TAG_NAME, value='td')
            if len(columns) == 5:  # Se for menor que isso é só info de quad
                row_data = [col.text for col in columns]
                data.append(row_data)

        subjects_professor_map = {}
        for sample in data:
            subject = sample[0].upper()  # Para manter o padrão com as tabelas Sigaa

            if sample[1] != '-':
                professor = sample[1]
            
            elif sample[2] != '-':
                professor = sample[2]

            else:
                professor = None
            
            subjects_professor_map[subject] = professor

        return subjects_professor_map
        

    def get_subject_grades_prop(self, subject_name: str, professor_name: str=None) -> dict:
        # Padroniza os nomes para conseguir fazer a busca no Next
        subject_name_std, subject_name_splitted = self.__standardize_name(subject_name)

        # Antes de mais nada, vai para página de reviews para procurar pelo professor ou matéria
        self.__driver.get('https://ufabcnext.com/app/reviews')

        # Procura pelo professor (se tiver sido informado) e uma matéria em específico
        if professor_name is not None:
            professor_name_std, professor_name_splitted = self.__standardize_name(professor_name)
            
            professors_input = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
                EC.presence_of_element_located((By.ID, 'input-15'))
            )
            professors_input.send_keys(professor_name_splitted)
            option = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
                # EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][contains(., '{professor_name_std.title()}')]"))  # Se o texto conter aquilo
                EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][text()=' {professor_name_std.title()}']"))  # Se o texto for igual! Tem sempre um espacinho na frente. Pode ser perigoso se houver uma pequena diferença!
            )
            option.click()

            # Aperta o ícone que abre o conjunto de matérias do professor
            icon = WebDriverWait(self.__driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//div[@class="v-field__append-inner"]//i[contains(@class, "mdi-menu-down")]'))
            )
            icon.click()

        else:
            subjects_input = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
                EC.presence_of_element_located((By.ID, 'input-15'))
            )

            subjects_input.send_keys(subject_name_splitted)
            subject_name_std = ' ' + subject_name_std  # Se for informar pelo input-15, precisa add um espaço antes...
        
        option = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            # EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][contains(., '{subject_name_std}')]"))  # Se o texto conter aquilo
            EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'v-list-item')][text()='{subject_name_std}']"))  # Se o texto for igual! Tem sempre um espacinho na frente. Pode ser perigoso se houver uma pequena diferença!
        )
        option.click()
       

        # Puxa os valores e imprime na tela
        grades_percentage = WebDriverWait(self.__driver, self.__max_time_to_wait).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, 'highcharts-text-outline'))
        )
        grades_dict = {gp[0]: gp[1].split('%')[0].strip() for gp in [gp_.text.split(':') for gp_ in grades_percentage]}

        return grades_dict
    

    def get_grades_df(self, subjects_list: Iterable, subjects_professor_map: dict={}) -> pd.DataFrame:
        subjects_dict = {}
        error_subjects_list = []
        
        for subject in tqdm(subjects_list):
            try:
                professor_name = subjects_professor_map.get(subject, None)
                grades_dict = self.get_subject_grades_prop(subject, professor_name)    
                subjects_dict[subject] = grades_dict

            except Exception:
                error_subjects_list.append(subject)

            finally:
                time.sleep(2)  # Para evitar de tomar cooldown

        df = pd.DataFrame.from_dict(subjects_dict).T.reset_index().rename(columns={'index': 'Disciplina'}).fillna(0)
        df['F'] = df['F'].astype(float) + df['O'].astype(float)
        df = df.drop(columns=['O'])
        df[['A', 'B', 'C', 'D', 'F']] = df[['A', 'B', 'C', 'D', 'F']].astype(float)
        df['Nota provável'] = df[['A', 'B', 'C', 'D', 'F']].apply(lambda s: (s['A']*4 + s['B']*3 + s['C']*2 + s['D']*1)/s[['A', 'B', 'C', 'D', 'F']].sum(), axis=1)
        df['Conceito provável'] = df['Nota provável'].apply(lambda s: 'F' if 0.0 <= s < 0.8 
                                                                            else 'D' if 0.8 <= s < 1.6
                                                                            else 'C' if 1.6 <= s < 2.4
                                                                            else 'B' if 2.4 <= s < 3.2
                                                                            else 'A')
        df['Conceito moda'] = df[['A', 'B', 'C', 'D', 'F']].apply(lambda s: np.argmax(s), axis=1).map({0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F'})
        
        for error_subject in error_subjects_list:
            df = pd.concat([df, pd.DataFrame({'Disciplina': [error_subject]})])
        
        df = df.reset_index(drop=True)

        return df
    

    def shutdown(self) -> None:
        self.__driver.quit()


In [10]:
credentials = {
    'e-mail': os.environ['EMAIL'],
    'ra': os.environ['RA']
}

subjects = df_ufabc['Disciplina'].values

scraper = UFABCNextGradeScraper(options=OPTIONS, credentials=credentials, max_time_to_wait=MAX_TIME_TO_WAIT)
scraper.login()
subjects_professor_map = scraper.get_subjects_professor_map()  # Se quiser puxar só a média por matéria (sem levar em conta professor), é só remover essa passagem
df_next = scraper.get_grades_df(subjects, subjects_professor_map)
scraper.shutdown()

df_next

  0%|          | 0/78 [00:00<?, ?it/s]

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
0,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,31.0,43.1,17.4,4.8,3.7,2.929000,B,B
1,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,84.6,15.4,0.0,0.0,0.0,3.846000,A,A
2,BASES COMPUTACIONAIS DA CIÊNCIA,39.2,26.9,18.3,7.2,8.4,2.813000,B,A
3,ESTRUTURA DA MATÉRIA,17.5,29.5,31.4,10.4,11.3,2.314685,C,C
4,BASES MATEMÁTICAS,8.7,15.4,25.8,15.5,34.5,1.482482,D,F
...,...,...,...,...,...,...,...,...,...
73,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,36.4,35.9,19.0,5.2,3.5,2.965000,B,A
74,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,60.0,26.7,0.0,0.0,13.3,3.201000,A,A
75,TEORIA DE ACIONAMENTOS ELÉTRICOS,26.1,30.4,22.1,9.3,12.0,2.493493,B,B
76,SISTEMAS DE CONTROLE II,6.4,32.1,34.6,15.4,11.6,2.062937,C,C


Antes de exportarmos os dados, vamos verificar as matérias que tivemos problema.

In [11]:
df_next[df_next.isna().any(axis=1)]

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
77,INSTRUMENTAÇÃO E METROLOGIA ÓPTICA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Como são poucas matérias, vamos corrigir o problema manualmente e, então, a base estará pronta para ser exportada.

In [12]:
a2f_percentages = [31.9, 25.9, 30.4, 4.4, 7.4]
probable_grade = sum([i*j for i, j in zip(range(1, 5), a2f_percentages)])/100
probable_concept = 'F' if 0.0 <= probable_grade < 0.8 else 'D' if 0.8 <= probable_grade < 1.6 else 'C' if 1.6 <= probable_grade < 2.4 else 'B' if 2.4 <= probable_grade < 3.2 else 'A'
mode_concept = ['A', 'B', 'C', 'D', 'F'][np.argmax(a2f_percentages)]

imo_values = np.concat([a2f_percentages, [probable_grade], [probable_concept], [mode_concept]])
imo_values

array(['31.9', '25.9', '30.4', '4.4', '7.4', '1.925', 'C', 'A'],
      dtype='<U32')

In [13]:
df_next.loc[df_next['Disciplina'] == 'INSTRUMENTAÇÃO E METROLOGIA ÓPTICA', 
            ['A', 'B', 'C', 'D', 'F', 'Nota provável', 'Conceito provável', 'Conceito moda']] = imo_values
df_next[df_next['Disciplina'] == 'INSTRUMENTAÇÃO E METROLOGIA ÓPTICA']

/var/folders/b1/0p0jycgj6llcx6cqndttv6180000gn/T/ipykernel_70093/2968863147.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '31.9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_next.loc[df_next['Disciplina'] == 'INSTRUMENTAÇÃO E METROLOGIA ÓPTICA',
/var/folders/b1/0p0jycgj6llcx6cqndttv6180000gn/T/ipykernel_70093/2968863147.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '25.9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_next.loc[df_next['Disciplina'] == 'INSTRUMENTAÇÃO E METROLOGIA ÓPTICA',
/var/folders/b1/0p0jycgj6llcx6cqndttv6180000gn/T/ipykernel_70093/2968863147.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '30.4' has dtype incompa

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
77,INSTRUMENTAÇÃO E METROLOGIA ÓPTICA,31.9,25.9,30.4,4.4,7.4,1.925,C,A


Finalmente, vamos ver se está tudo certo e, em caso positivo, vamos exportar os dados.

In [14]:
df_next[df_next.isna().any(axis=1)]

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda


In [15]:
df_next.to_csv('./data/notas-next.csv', sep=';', index=False)